# 三、情感分析

### 情感分析含义

情感分析指的是对新闻报道、商品评论、电影影评等文本信息进行观点提取、主题分析、情感挖掘。情感分析常用于对某一篇新闻报道积极消极分析、淘宝商品评论情感打分、股评情感分析、电影评论情感挖掘。情感分析的内容包括:情感的持有者分析、态度持有者分析、态度类型分析（一系列类型如喜欢（like），讨厌（hate），珍视（value），渴望（desire）等；或着简单的加权极性如积极（positive），消极（negative）和中性（neutral）并可用具体的权重修饰）、态度的范围分析（包含每句话，某一段、或者全文）。

### 情感分析方法

情感挖掘目前主要使用的方法是使用情感词典，对文本进行情感词匹配，汇总情感词进行评分，最后得到文本的情感倾向。通常有两种方法进行情感分析。

第一种：基于BosonNLP情感词典。该情感词典是由波森自然语言处理公司推出的一款已经做好标注的情感词典。BosonNLp情感词典下载地址：https://bosonnlp.com/dev/resource 。基于BosonNLP情感词典的情感分析较为简单。
    
    首先，需要对文本进行分句、分词，本文选择的分词工具为哈工大的pyltp。

    其次，将分词好的列表数据对应BosonNLp词典进行逐个匹配，并记录匹配到的情感词分值。

    最后，统计计算分值总和，如果分值大于0，表示情感倾向为积极的；如果小于0，则表示情感倾向为消极的。
    

第二种：采用的是知网推出的情感词典，以及极性表进行情感分析。知网提供的情感词典共用12个文件，分为英文和中文。其中中文情感词典包括：评价、情感、主张、程度（正面、负面）的情感文本。本文将评价和情感词整合作为情感词典使用，程度词表中含有的程度词，按照等级区分，分为：most（最高）-very（很、非常）-more（更多、更）-ish（稍、一点点）-insufficiently（欠、不）-over（过多、多分、多）六个情感程度词典。知网情感词典下载地址：- http://www.keenage.com/html/c_bulletin_2007.htm


基于知网情感词典的情感分析原理分为以下几步：

    1、首先，需要对文本分句，分句，得到分词分句后的文本语料，并将结果与哈工大的停用词表比对，去除停用词；
    
    哈工大语言技术平台LTP(Language Technology Platform)提供了一系列中文自然语言处理工具，用户可以使用这些工具对于中文文本进行分词、词性标注、句法分析等等工作。LTP本身是基于C++开发的，不过官方也提供了Python的封装包--pyltp，whl下载地址为：https://pypi.tuna.tsinghua.edu.cn/simple/pyltp-binary/ ，使用说明：https://pyltp.readthedocs.io/zh_CN/latest/api.html#id2

    2、其次，对每一句话进行情感分析，分析的方法主要为：判断这段话中的情感词数目，含有积极词，则积极词数目加1，含有消极词，则消极词数目加1。并且再统计的过程中还需要判断该情感词前面是否存在程度副词，如果存在，则需要根据程度副词的种类赋予不同的权重，乘以情感词数。如果句尾存在？！等符号，则情感词数目增加一定值，因为！与？这类的标点往往表示情感情绪的加强，因此需要进行一定处理。

    3、接着统计计算整段话的情感值(积极词值-消极词值），得到该段文本的情感倾向。

    4、最后，统计每一段的情感值，相加得到文章的情感值。


### 甄嬛传剧本情感分析

我们将对五位主角（甄嬛、皇后、皇帝、华妃/华贵妃/年妃/年答应、安陵容）每集的情感做分析，并将结果可视化。

In [1]:
import pyltp
from pyltp import Segmentor
from pyltp import SentenceSplitter
from pyltp import Postagger

import os
import sys
import re

import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# -*- coding: utf-8 -*-

#写入文件并把剧本分集，一共有76集，chapter[1::2]则是每集对应的剧本
with  open("C:/Users/guoqs/Desktop/script.txt", 'r',encoding='utf-8')as f:
    script = f.read()
pattern = r'(第[\u4e00-\u9fa5]{1,3}集)'
chapter = [c for c in re.split(pattern, script) if c]
print(chapter[::2])
chapter = chapter[1::2] 

['第一集', '第二集', '第三集', '第四集', '第五集', '第六集', '第七集', '第八集', '第九集', '第十集', '第十一集', '第十二集', '第十三集', '第十四集', '第十五集', '第十六集', '第十七集', '第十八集', '第十九集', '第二十集', '第二十一集', '第二十二集', '第二十三集', '第二十四集', '第二十五集', '第二十六集', '第二十七集', '第二十八集', '第二十九集', '第三十集', '第三十一集', '第三十二集', '第三十三集', '第三十四集', '第三十五集', '第三十六集', '第三十七集', '第三十八集', '第三十九集', '第四十集', '第四十一集', '第四十二集', '第四十三集', '第四十四集', '第四十五集', '第四十六集', '第四十七集', '第四十八集', '第四十九集', '第五十集', '第五十一集', '第五十二集', '第五十三集', '第五十四集', '第五十五集', '第五十六集', '第五十七集', '第五十八集', '第五十九集', '第六十集', '第六十一集', '第六十二集', '第六十三集', '第六十四集', '第六十五集', '第六十六集', '第六十七集', '第六十八集', '第六十九集', '第七十集', '第七十一集', '第七十二集', '第七十三集', '第七十四集', '第七十五集', '第七十六集']


In [3]:
#对每一集中的主要角色台词划分并存储
def distrib(text):
    #把一集的剧本由字符串返回list类型
    data = text.split('\n')
    #建立5个主角的台词库
    ZhenHuan = ''
    HuangDi = ''
    HuangHou = ''
    HuaFei = ''
    LingRong = ''
    for each_line in data:
        try:
            (role,line_spoken)  =each_line.split('：',1)
            line_spoken = line_spoken.strip()
            if role[:2] == '甄嬛':
                ZhenHuan += line_spoken
            elif role[:2] == '皇帝':
                HuangDi += line_spoken
            elif role[:2] == '皇后':
                HuangHou += line_spoken
            elif role[:2] == '华妃' or role[:2] == '年妃' or role[:3] == '华贵妃' or role[:3] == '华答应':
                HuaFei += line_spoken
            elif role[:2] == '陵容' or role[:3] == '安陵容':
                LingRong += line_spoken
        except ValueError:
            pass
    return [ZhenHuan, HuangDi, HuangHou, HuaFei, LingRong]

In [4]:
#文本分句
def cut_sentence(text):
    sentences = SentenceSplitter.split(text)
    sentence_list = [ w for w in sentences]
    return sentence_list
 
#文本分词
def tokenize(sentence):
    #加载模型
    segmentor = Segmentor()  # 初始化实例
    # 加载模型
    segmentor.load('C:/ProgramData/Anaconda3/Lib/site-packages/pyltp_binary-0.2.1.0.dist-info/ltp_data_v3.4.0/cws.model')
    # 产生分词，segment分词函数
    words = segmentor.segment(sentence)
    words = list(words)
    # 释放模型
    segmentor.release()
    return words
 
#词性标注
def postagger(words):
    # 初始化实例
    postagger = Postagger()
    # 加载模型
    postagger.load('C:/ProgramData/Anaconda3/Lib/site-packages/pyltp_binary-0.2.1.0.dist-info/ltp_data_v3.4.0/pos.model')
    # 词性标注
    postags = postagger.postag(words)
    # 释放模型
    postagger.release()
    #返回list
    postags = [i for i in postags]
    return postags
 
# 分词，词性标注，词和词性构成一个元组
def intergrad_word(words, postags):
    #拉链算法，两两匹配
    pos_list = zip(words, postags)
    pos_list = [ w for w in pos_list]
    return pos_list

#读取文件，文件读取函数
def read_file(filename, enc):
    with  open(filename, 'r', encoding=enc) as f:
        text = f.read()
        #返回list类型数据
        text = text.split('\n')
    return text
     
#去停用词函数
def del_stopwords(words):
    # 读取停用词表
    stopwords = read_file("C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/stopwords/中文停用词库.txt", 'utf-16')
    # 去除停用词后的句子
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words
   
#读取情感词典
posdict = read_file('C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/台湾大学NTUSD/NTUSD_positive_simplified.txt', 'utf-16')
negdict = read_file('C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/台湾大学NTUSD/NTUSD_negative_simplified.txt', 'utf-16')

# 读取程度副词词典
# 权值为2
mostdict = read_file("C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/知网Hownet/1most.txt", 'utf-8')
# 权值为1.75
verydict =  read_file("C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/知网Hownet/2very.txt", 'utf-8')
# 权值为1.50
moredict = read_file("C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/知网Hownet/3more.txt", 'utf-8')
# 权值为1.25
ishdict = read_file("C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/知网Hownet/4ish.txt", 'utf-8')
# 权值为0.25
insufficientdict =  read_file("C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/知网Hownet/5insufficiently.txt", 'utf-8')
# 权值为-1
inversedict = read_file("C:/Users/guoqs/Desktop/data_analysis/HW/期末大作业/dict/emotion/其他词典和分类/否定词典/否定.txt", 'ANSI')
 
#程度副词处理，对不同的程度副词给予不同的权重
def match_adverb(word, sentiment_value):
    #最高级权重为
    if word in mostdict:
        sentiment_value *= 8
    #比较级权重
    elif word in verydict:
        sentiment_value *= 6
    #比较级权重
    elif word in moredict:
        sentiment_value *= 4
    #轻微程度词权重
    elif word in ishdict:
        sentiment_value *= 2
    #相对程度词权重
    elif word in insufficientdict:
        sentiment_value *= 0.5
    #否定词权重
    elif word in inversedict:
        sentiment_value *= -1
    else:
        sentiment_value *= 1
    return sentiment_value

In [5]:
#对一段台词进行情感打分
def single_sentiment_score(text_sent):
    sentiment_scores = []
    #对台词分句
    sentences = cut_sentence(text_sent)
    for sent in sentences:
        #分词
        words = tokenize(sent)
        seg_words = del_stopwords(words)
        #i，s 记录情感词和程度词出现的位置
        i = 0   #记录扫描到的词位子
        s = 0   #记录情感词的位置
        poscount = 0 #记录积极情感词数目
        negcount = 0 #记录消极情感词数目
        #逐个查找情感词
        for word in seg_words:
            #如果为积极词
            if word in posdict:
                poscount += 1  #情感词数目加1
            #在情感词前面寻找程度副词
                for w in seg_words[s:i]:
                    poscount = match_adverb(w, poscount)
                s = i+1 #记录情感词位置
            # 如果是消极情感词
            elif word in negdict:
                negcount +=1
                for w in seg_words[s:i]:
                    negcount = match_adverb(w,negcount)
                s = i+1
            #如果结尾为感叹号或者问号，表示句子结束，并且倒序查找感叹号前的情感词，权重+4
            elif word =='!' or  word =='！' or word =='?' or word == '？':
                for w2 in seg_words[::-1]:
                    #如果为积极词，poscount+2
                    if w2 in posdict:
                        poscount += 4
                        break
                    #如果是消极词，negcount+2
                    elif w2 in negdict:
                        negcount += 4
                        break
            i += 1 #定位情感词的位置
        #计算情感值
        sentiment_score = poscount - negcount
        sentiment_scores.append(sentiment_score)
    #计算这段台词的情感总得分
    sentiment_sum = sum(sentiment_scores)
    return sentiment_sum

In [7]:
#生成列表以建立DataFrame，每一个列表元素是一个字典，包含第几集、主角、情感评分三个信息
data = []
character = ['甄嬛', '皇帝', '皇后', '华妃', '陵容']
#n表示第几集
for n in range(1):
    text = distrib(chapter[n])
    for i in range(5):
        #考虑到部分主角可能在一集中没有台词
        if text[i] != '':
            dict = {}
            dict["集数"] = n+1
            dict['角色'] = character[i]
            dict['情感得分'] = single_sentiment_score(text[i])
            data.append(dict)
df = pd.DataFrame(data)
print(df)

   集数  角色  情感得分
0   1  甄嬛   -17
1   1  皇帝     2
2   1  皇后     7
3   1  华妃     1
4   1  陵容    27
